In [1]:
import pandas as pd
import functions as fn

In [ ]:
### TODO
### process discharge summaries to only include text sections
### iterate through entire dataframe, extract all discharge summaries, save as text file

Questions:
- Should this be a NER task?
- If it is a NER task - would it be: can I identify which symptoms are potentially related to irAEs?
- Should there be any additional considerations? e.g. which files belong to people with chemo, etc
- removing:
    - everything above HOPC
    - everything below Discharge Instructions

In [2]:
file_path = "./data/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/note/"

In [3]:
df = pd.read_csv(file_path+"discharge.csv")

In [6]:
test = df["text"][0]

In [31]:
def clean_report(report):
    start_marker = "History of Present Illness:"
    end_marker = "Discharge Instructions:"
    start_index = report.find(start_marker)
    end_index = report.find(end_marker)
    if start_index == -1:
        start_index = 0
    if end_index == -1:
        end_index = len(report)
    return report[start_index:end_index].strip()

def card_generator(input_string, n=5):
    # reformats text breaks to make annotation cards more consistent in length
    sentences = input_string.replace("\n", "")
    sentences = sentences.split(".")
    result = ""
    for i in range(len(sentences)):
        result += sentences[i]
        if (i+1) % n == 0 and i < len(sentences)-1:
            result += ". \n"
        else:
            result += ". "
    result += "\n"
    return result 

In [34]:
text = ""
for i in range(len(df)):
    simplified_text = clean_report(df["text"][i])
    cards = card_generator(simplified_text)
    text += cards
text
with open("./data/processed_text.txt", "w") as f:
    f.write(text)

In [6]:
dir = "./data"
label_list = [
   "Inflammatory_symptom/sign",
   "Inflammatory_investigation_results",
   "Cancer",
   "Immunotherapy",
   "Chemotherapy" 
]
database = "irAE"
model = "en_core_web_sm"
annotate = fn.AnnotationProcess(dir, label_list, database, model)

annotate.database_connect()

annotate.prodigy_connect("ner.manual", "./data/processed_text.txt")


Using 5 label(s): Inflammatory_symptom/sign, Inflammatory_investigation_results,
Cancer, Immunotherapy, Chemotherapy

✨  Starting the web server at http://localhost:8080 ...
Open the app in your browser and start annotating!



In [25]:
t = "this is a sentence. \n this is another sentence"
sentences = t.replace("\n", "")
sentences = sentences.split(". ")
sentences

['this is a sentence', ' this is another sentence']